In [ ]:
#### distances and angles D-A
import os
from rdkit import Chem
import numba


import MDAnalysis
from MDAnalysis import *
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import pandas as pd
import numpy as np
import math
from MDAnalysis.analysis.align import *
from MDAnalysis.analysis.rms import rmsd
from MDAnalysis.analysis.rdf import InterRDF
import MDAnalysis as mda
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as mpatches
import pandas as pd


numdir='./numbering_dihedral/'
mol2dir='./mol2_basal/'
scan='./min_basal_xyz/'
indir='./min_basal_mol/'
outdir='./out_run/'

for root, dirs, filenames in os.walk(scan):
    for n in filenames:
        print(n)
        if n[-10:-4] in ['_hexil']:
            m = Chem.MolFromMolFile(indir+n[:-10]+'.mol',removeHs=True)
        else:
            m = Chem.MolFromMolFile(indir+n[:-4]+'.mol',removeHs=True)
        bond_idx=[]
        
        r7=open(numdir+n[:-4]+'.dat', 'r')
        table7 = [line.rstrip().split('-') for line in r7.readlines()]
        if n[:-4] in ['C342','C385']:
            for line in range(0,int(len(table7)/4)):
                bond_idx.append(m.GetBondBetweenAtoms(int(table7[line*4][1])-2,int(table7[line*4][2])-2).GetIdx())
        
        else:
            for line in range(0,int(len(table7)/4)):
                bond_idx.append(m.GetBondBetweenAtoms(int(table7[line*4][1])-1,int(table7[line*4][2])-1).GetIdx())
                    
                    
                    
      
        fragmented_mol = Chem.FragmentOnBonds(m, bond_idx,addDummies=False)    
        
        smile=Chem.MolToSmiles(fragmented_mol)
        
        table7=smile.split('.')
        
        
        NO=[]
        MM=[]
        FRAG=[]
        for index,i in enumerate(table7):
            
            m1=Chem.MolFromSmiles(table7[index])
            MM.append(m.GetSubstructMatches(m1))
        for i in MM:
            for j in i:
                if j not in FRAG:
                    FRAG.append(j)
        
        FRAG1=[]
        for i in range(0,len(FRAG)):
            for j in range(0,len(FRAG)):
                if j>i:
                    if set(FRAG[i]) & set(FRAG[j]) != set():
                        if len(FRAG[i])>len(FRAG[j]):
                            FRAG1.append(FRAG[j])
        FRAG3=[] 
        for i in FRAG:
                if i not in FRAG1:
                    FRAG3.append(i)
        
        
        NAMES=[]
        
        for j in FRAG3:
            names=[]
            lista=[]
            for l in j:
                lista.append(l)
            for i in lista:
                mol2file=open(mol2dir+n[:-4]+'.mol2','r')
                for line in mol2file:
                    if len(line)>70:
                        a=[line.rstrip().split(" ") for line in line]
                        b=str(a[4][0]+a[5][0]+a[6][0])
                        if str(i+1)==b:
                            names.append(str(a[8][0]+''+a[9][0]+''+a[10][0]+''+a[11][0]))
            NO.append(names)                           
            no=' '.join(names)+' '
            NAMES.append(no)
        print(NAMES)
        if outdir == './out_run/':
            frame=str(253)
        elif outdir == './out_c/':     
            frame=str(164)
        
        traj=open(outdir+n[:-4]+'/traj.in', 'w')
        traj.write('parm pcbm_LIG2.prmtop\n')
        traj.write('trajin traj.nc '+frame+' last 10\n')
        traj.write('trajout p1.mdcrd\n')
        traj.write('run\n')
        traj.close()
        cmd='cd '+outdir+n[:-4]+'/ && cpptraj -i traj.in && rm traj.in'
        os.system(cmd)
        
        u = mda.Universe(outdir+n[:-4]+"/pcbm_LIG2.prmtop",outdir+n[:-4]+"/p1.mdcrd")
        
        
        sel_aceptor = "(resname PCB and not name H* C11 C10 C9 C8 C12 C7 C6 C1 C2 O1 O2 C3 C5 C4)"
    
        SELE=[]
        for i in NAMES:
            SELE.append("resname LIG and name "+str(i)+" ")
        
        data=("resname LIG and name "+" ".join([ str(x) for x in NAMES]))
        
        
        donor = u.select_atoms(data)
        aceptor = u.select_atoms(sel_aceptor)
       
        donor_list=list(donor.residues)
        aceptor_list=list(aceptor.residues)
        ANGLE = []
        menor_dist=12.0
        DIST = []
        
        prom_ang=[]
        prom_dis=[]
        for ts in u.trajectory:
            #print(ts)
            DDDDD=[]
            AAAAA=[]
            for r in aceptor_list: 
                for r2 in donor_list:
                    r1=r.select_atoms(sel_aceptor)
                    CM_1=np.array([r1.center_of_geometry()])
                    
                    CM_2=r2.select_atoms(data)
                    CM_2=np.array([CM_2.center_of_geometry()])
                    CM0=MDAnalysis.lib.distances.distance_array(CM_1, CM_2)
                    CM0=float(CM0)
                    
                    for index,i in enumerate(SELE):
                        core2=r2.select_atoms(SELE[index])
                        centro2=np.array([core2.center_of_geometry()])
                        dist = MDAnalysis.lib.distances.distance_array(CM_1, centro2)
                        dist=float(dist)
                        if dist < menor_dist:
                                    p1=np.array(r2.select_atoms('name '+NO[index][0]).center_of_geometry())
                                    p2=np.array(r2.select_atoms('name '+NO[index][1]).center_of_geometry())
                                    p3=np.array(r2.select_atoms('name '+NO[index][3]).center_of_geometry())
                                    v1=p2-p1
                                    v2=p3-p1
                                    normal=np.cross(v1,v2)
                                    vector=centro2-CM_1
                                    vector=vector[0]
                                    long1=np.sqrt(np.dot(vector,vector))
                                    long2=np.sqrt(np.dot(normal,normal))
                                    dot=np.dot(normal,vector)
                                    angle=(np.arccos(dot/long1/long2))*360/np.pi
                                    if angle <= 90:
                                        angle = angle
                                        ANGLE.append(angle)
                                        AAAAA.append(angle)
                                    elif 180 >= angle > 90:
                                        angle = 180-angle
                                        ANGLE.append(angle)
                                        AAAAA.append(angle)
                                    elif  270>= angle > 180:
                                        angle = angle-180
                                        ANGLE.append(angle)
                                        AAAAA.append(angle)
                                    else:
                                        angle = 360-angle
                                        ANGLE.append(angle)
                                        AAAAA.append(angle)
                                    DIST.append(CM0)
                                    DDDDD.append(CM0)
            prom_ang.append(sum(AAAAA)/len(AAAAA))
            prom_dis.append(sum(DDDDD)/len(DDDDD))
        print(len(ANGLE))
        print(len(DIST))
        
        print(str(n[:-4]),'ang '+str(np.average(np.array(prom_ang)))+'+-'+str(np.std(np.array(prom_ang))),'dist '+str(np.average(np.array(prom_dis)))+'+-'+str(np.std(np.array(prom_dis))))
        
        data=open(outdir+n[:-4]+'/'+'angle_cm_da.txt', 'w')
        for index,yy in enumerate(ANGLE):
            data.write("{:12f}  {:12f}  ".format(float(DIST[index]),float(ANGLE[index]))+'\n')
        data.close()        
        
        cmd='cd '+outdir+n[:-4]+'/ && rm p1.mdcrd'
        os.system(cmd)
        
        sns.set(font_scale=1.5)
        plt.gcf().subplots_adjust(bottom=0.15)
        sns.axes_style("white")
        sns.set_style("ticks", {'figure.facecolor': "white"})
        
        t = sns.JointGrid(np.array(DIST), np.array(ANGLE) ,data=None,xlim=(4,20.0),ylim=(0,90)).set_axis_labels("Distancia$_{moléculas}$ ($\AA$)","$\Theta$$_{anillos}$ (Grados)")
        t= t.plot_joint(sns.kdeplot, cmap="Blues", shade=True, shade_lowest=False)
        t = t.plot_marginals(sns.kdeplot, color="blue", shade=True, shade_lowest=False)
        t.savefig(outdir+n[:-4]+'/'+n[:-4]+'_a.png')
        plt.show()

In [ ]:
#### distances and angles D-A
import MDAnalysis
from MDAnalysis import *
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import pandas as pd
import numpy as np
import math
from MDAnalysis.analysis.align import *
from MDAnalysis.analysis.rms import rmsd
from MDAnalysis.analysis.rdf import InterRDF
import MDAnalysis as mda
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as mpatches
import pandas as pd
sns.set(font_scale=1.5)
plt.gcf().subplots_adjust(bottom=0.15)
sns.axes_style("white")
sns.set_style("ticks", {'figure.facecolor': "white"})
outdir='./out_run/'
n='C271_hexil.xyz'


data=open(outdir+n[:-4]+'/'+'angle_cm_da.txt', 'r')
table = [line.rstrip().split() for line in data.readlines()]
DIST=[]
ANGLE=[]

for i in table:
    DIST.append(i[0])
    ANGLE.append(i[1])

       
t = sns.JointGrid(np.array(DIST), np.array(ANGLE) ,data=None,xlim=(4,20.0),ylim=(0,90)).set_axis_labels("$d_{da}$ ($\AA$)","$\\theta_{da}$ (Degree)")
t= t.plot_joint(sns.kdeplot, cmap="Blues", shade=True, shade_lowest=False)

t = t.plot_marginals(sns.kdeplot, color="blue", shade=True, shade_lowest=False)
t.savefig(outdir+n[:-4]+'/'+n[:-4]+'_a.png')
plt.show()

In [ ]:
##### distances and angles D-D
import os
from rdkit import Chem

import MDAnalysis
from MDAnalysis import *
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import pandas as pd
import numpy as np
import math
from MDAnalysis.analysis.align import *
from MDAnalysis.analysis.rms import rmsd
from MDAnalysis.analysis.rdf import InterRDF
import MDAnalysis as mda
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as mpatches
import pandas as pd
import seaborn as sns; sns.set(color_codes=True)

scan='./min_basal_xyz/'
indir='./min_basal_mol/'
outdir='./out_run/'
numdir='./numbering_dihedral/'
mol2dir='./mol2_basal/'

for root, dirs, filenames in os.walk(scan):
    for n in filenames:
        print(n)
        if n[-10:-4] in ['_hexil']:
            m = Chem.MolFromMolFile(indir+n[:-10]+'.mol',removeHs=True)
        else:
            m = Chem.MolFromMolFile(indir+n[:-4]+'.mol',removeHs=True)
        bond_idx=[]
        
        r7=open(numdir+n[:-4]+'.dat', 'r')
        table7 = [line.rstrip().split('-') for line in r7.readlines()]
        if n[:-4] in ['C342','C385']:
            for line in range(0,int(len(table7)/4)):
                bond_idx.append(m.GetBondBetweenAtoms(int(table7[line*4][1])-2,int(table7[line*4][2])-2).GetIdx())
        
        else:
            for line in range(0,int(len(table7)/4)):
                bond_idx.append(m.GetBondBetweenAtoms(int(table7[line*4][1])-1,int(table7[line*4][2])-1).GetIdx())
                    
                    
                    
      
        fragmented_mol = Chem.FragmentOnBonds(m, bond_idx,addDummies=False)    
        
        smile=Chem.MolToSmiles(fragmented_mol)
        
        table7=smile.split('.')
        
        
        NO=[]
        MM=[]
        FRAG=[]
        for index,i in enumerate(table7):
            
            m1=Chem.MolFromSmiles(table7[index])
            MM.append(m.GetSubstructMatches(m1))
        for i in MM:
            for j in i:
                if j not in FRAG:
                    FRAG.append(j)
        
        FRAG1=[]
        for i in range(0,len(FRAG)):
            for j in range(0,len(FRAG)):
                if j>i:
                    if set(FRAG[i]) & set(FRAG[j]) != set():
                        if len(FRAG[i])>len(FRAG[j]):
                            FRAG1.append(FRAG[j])
        FRAG3=[] 
        for i in FRAG:
                if i not in FRAG1:
                    FRAG3.append(i)
        
        
        NAMES=[]
        
        for j in FRAG3:
            names=[]
            lista=[]
            for l in j:
                lista.append(l)
            for i in lista:
                mol2file=open(mol2dir+n[:-4]+'.mol2','r')
                for line in mol2file:
                    if len(line)>70:
                        a=[line.rstrip().split(" ") for line in line]
                        b=str(a[4][0]+a[5][0]+a[6][0])
                        if str(i+1)==b:
                            names.append(str(a[8][0]+''+a[9][0]+''+a[10][0]+''+a[11][0]))
            NO.append(names)                           
            no=' '.join(names)+' '
            NAMES.append(no)
        print(NAMES)
        if outdir == './out_run/':
            frame=str(2530)
        elif outdir == './out_c/':     
            frame=str(820)
        
        traj=open(outdir+n[:-4]+'/traj.in', 'w')
        traj.write('parm pcbm_LIG2.prmtop\n')
        traj.write('trajin traj.nc '+frame+' last 10\n')
        traj.write('trajout p1.mdcrd\n')
        traj.write('run\n')
        traj.close()
        cmd='cd '+outdir+n[:-4]+'/ && cpptraj -i traj.in && rm traj.in'
        os.system(cmd)
        
        u = mda.Universe(outdir+n[:-4]+"/pcbm_LIG2.prmtop",outdir+n[:-4]+"/p1.mdcrd")
        
        sel_aceptor = "(resname PCB and not name H* C11 C10 C9 C8 C12 C7 C6 C1 C2 O1 O2 C3 C5 C4)"
        
        SELE=[]
        for i in NAMES:
            SELE.append("resname LIG and name "+str(i)+" ")
        
        data=("resname LIG and name "+" ".join([ str(x) for x in NAMES]))
        
        
        donor = u.select_atoms(data)
        aceptor = u.select_atoms(sel_aceptor)
       
        donor_list=list(donor.residues)
        aceptor_list=list(aceptor.residues)
        ANGLE = []
        menor_dist=6.0
        DIST = []
        
        prom_ang=[]
        prom_dis=[]
        for ts in u.trajectory:
            print(ts)
            #print(ts)
            DDDDD=[]
            AAAAA=[]
            for r in donor_list:
                for r2 in donor_list:
                    CORE=[]
                    if r2 != r:
                        CM_1=r.select_atoms(data)
                        CM_1=np.array([CM_1.center_of_geometry()])
                        CM_2=r2.select_atoms(data)
                        CM_2=np.array([CM_2.center_of_geometry()])
                        CM0=MDAnalysis.lib.distances.distance_array(CM_1, CM_2)
                        CM0=float(CM0)
                        for index,i in enumerate(SELE):
                            
                            core=r.select_atoms(i)
                            centro=np.array([core.center_of_geometry()])
                            for index2,j in enumerate(SELE):
                                core2=r2.select_atoms(SELE[index2])
                                centro2=np.array([core2.center_of_geometry()])
                                dist = MDAnalysis.lib.distances.distance_array(centro, centro2)
                                dist=float(dist)
                                
                                if dist < menor_dist:
                                    p1=np.array(r.select_atoms('name '+NO[index][0]).center_of_geometry())
                                    p2=np.array(r.select_atoms('name '+NO[index][1]).center_of_geometry())
                                    p3=np.array(r.select_atoms('name '+NO[index][3]).center_of_geometry())
                                    v1=p2-p1
                                    v2=p3-p1
                                    normal=np.cross(v1,v2)
                                
                                    p4=np.array(r2.select_atoms('name '+NO[index2][0]).center_of_geometry())
                                    p5=np.array(r2.select_atoms('name '+NO[index2][1]).center_of_geometry())
                                    p6=np.array(r2.select_atoms('name '+NO[index2][3]).center_of_geometry())
                                    v3=p5-p4
                                    v4=p6-p4
                                    normal1=np.cross(v3,v4)  

                                    long1=np.sqrt(np.dot(normal,normal))
                                    long2=np.sqrt(np.dot(normal1,normal1))
                                    dot=np.dot(normal,normal1)
                                    angle=(np.arccos(dot/long1/long2))*360/np.pi
                                    if angle <= 90:
                                        angle = angle
                                        ANGLE.append(angle)
                                        AAAAA.append(angle)
                                    elif 180 >= angle > 90:
                                        angle = 180-angle
                                        ANGLE.append(angle)
                                        AAAAA.append(angle)
                                    elif  270>= angle > 180:
                                        angle = angle-180
                                        ANGLE.append(angle)
                                        AAAAA.append(angle)
                                    else:
                                        angle = 360-angle
                                        ANGLE.append(angle)
                                        AAAAA.append(angle)
                                    DIST.append(CM0)
                                    DDDDD.append(CM0)
                        
            prom_ang.append(sum(AAAAA)/len(AAAAA))
            prom_dis.append(sum(DDDDD)/len(DDDDD))
        print(len(ANGLE))
        print(len(DIST))
        
        print(str(n[:-4]),'ang '+str(np.average(np.array(prom_ang)))+'+-'+str(np.std(np.array(prom_ang))),'dist '+str(np.average(np.array(prom_dis)))+'+-'+str(np.std(np.array(prom_dis))))
        
        data=open(outdir+n[:-4]+'/'+'angle_cm_dd.txt', 'w')
        for index,yy in enumerate(ANGLE):
            data.write("{:12f}  {:12f}  ".format(float(DIST[index]),float(ANGLE[index]))+'\n')
        data.close()        
        
        cmd='cd '+outdir+n[:-4]+'/ && rm p1.mdcrd'
        os.system(cmd)
        
        
        sns.set(font_scale=1.5)
        plt.gcf().subplots_adjust(bottom=0.15)
        sns.axes_style("white")
        sns.set_style("ticks", {'figure.facecolor': "white"})
        
        t = sns.JointGrid(np.array(DIST), np.array(ANGLE) ,data=None,xlim=(0,20.0),ylim=(0,90)).set_axis_labels("Distancia$_{moléculas}$ ($\AA$)","$\Theta$$_{anillos}$ (Grados)")
        t= t.plot_joint(sns.kdeplot, cmap="Reds", shade=True, shade_lowest=False)
        t = t.plot_marginals(sns.kdeplot, color="red", shade=True, shade_lowest=False)
        t.savefig(outdir+n[:-4]+'/plano_'+n[:-4]+'_dd.png')
        plt.show()

In [ ]:
##### distances and angles D-D
import MDAnalysis
from MDAnalysis import *
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import pandas as pd
import numpy as np
import math
from MDAnalysis.analysis.align import *
from MDAnalysis.analysis.rms import rmsd
from MDAnalysis.analysis.rdf import InterRDF
import MDAnalysis as mda
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as mpatches
import pandas as pd

outdir='./out_c/'
n='C271_hexil.xyz'


data=open(outdir+n[:-4]+'/'+'angle_cm_dd.txt', 'r')
table = [line.rstrip().split() for line in data.readlines()]
DIST=[]
ANGLE=[]

for i in table:
    DIST.append(i[0])
    ANGLE.append(i[1])
        
        
sns.set(font_scale=1.5)
plt.gcf().subplots_adjust(bottom=0.15)
sns.axes_style("white")
sns.set_style("ticks", {'figure.facecolor': "white"})
        
t = sns.JointGrid(np.array(DIST), np.array(ANGLE) ,data=None,xlim=(0,20.0),ylim=(0,90)).set_axis_labels("$d_{dd}$ ($\AA$)","$\\theta$$_{dd}$ (Grados)")
t= t.plot_joint(sns.kdeplot, cmap="Reds", shade=True, shade_lowest=False)
t = t.plot_marginals(sns.kdeplot, color="red", shade=True, shade_lowest=False)
t.savefig(outdir+n[:-4]+'/plano_'+n[:-4]+'_dd.png')
plt.show()